In [48]:
with open ('./Chapter/Chapter-2.txt', 'r', encoding = 'utf-8-sig') as inFile:
    Lines = inFile.readlines()

In [51]:
Del = ['2.1','2.2','2.3','2.4','2.5','2.6']
for lid in range(len(Lines)):
    for d in Del:
        if d in Lines[lid]:
            Lines.remove(Lines) #要先打Lines[lid]執行過

In [52]:
for sid in range(len(Lines)):
    if len(Lines[sid])<=20 and Lines[sid] !='\n' :
        print(Lines[sid])
        Lines[sid]=''
while '' in Lines:
    Lines.remove('')

Key Point

Key Point

Key Point

Key Point

Key Point



In [53]:
text = ''.join(Lines)

In [54]:
import re;
text=text.replace('\n\n\n\n', '\n\n')
p=re.compile('\n\n',re.S)
paraList=p.split(text)

In [55]:
len(paraList)

118

In [56]:
import TSF2

In [57]:
K = 2 # the number of sentences in a block
THRESHOLD_DISSIM = 0.4 #根據Kern and Granitzer (2009)的建議訂為0.7。
ParaSenList = TSF2.SegementPara(paraList, K, THRESHOLD_DISSIM)

Avg = -0.09032341682390929
Dev = 1.2614359142391274


In [58]:
for i in range(len(ParaSenList)):
    print (i,len(ParaSenList[i]))
print('------')
print(len(ParaSenList))

0 4
1 13
2 6
3 2
4 5
5 2
6 1
7 8
8 2
9 14
10 2
11 2
12 2
13 2
14 4
15 2
16 4
17 2
18 4
19 2
20 3
21 1
22 17
23 2
24 2
25 2
26 4
27 4
------
28


In [61]:
for i in range(len(ParaSenList)):
    if len(ParaSenList[i]) <=2 : #要執行三遍
        ParaSenList[i-1].extend(ParaSenList[i])
        try:
            ParaSenList.remove(ParaSenList[i])
        except ValueError:
            pass

In [62]:
for i in range(len(ParaSenList)):
    print (i,len(ParaSenList[i]))
print('------')
print(len(ParaSenList))

0 4
1 13
2 8
3 8
4 10
5 16
6 6
7 6
8 6
9 6
10 4
11 19
12 4
13 4
14 4
------
15


In [63]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import numpy as np

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [64]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('averaged_perceptron_tagger')
nltk.download("wordnet")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/chenyifan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chenyifan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [65]:
doc_set=[]
texts = []

In [66]:
for k in range(len(ParaSenList)):
    str1 = ' '.join(ParaSenList[k])
    doc_set.append(str1)

In [67]:
len(doc_set)

15

In [68]:
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    #仅保留名词或特定POS   
    refiltered =nltk.pos_tag(stopped_tokens)
    filtered = [w for w, pos in refiltered if pos.startswith('NN') or pos.startswith('NNP')]
    
    # stem tokens
    lemma = nltk.wordnet.WordNetLemmatizer()    
    stemmed_tokens = [lemma.lemmatize(i, pos='n') for i in filtered]
    stemmed_tokens = [lemma.lemmatize(i, pos='v') for i in stemmed_tokens]

    for k in range(len(stemmed_tokens)):
        if len(stemmed_tokens[k])<3 :
            stemmed_tokens[k]=''
    while '' in stemmed_tokens:
        stemmed_tokens.remove('')
            
    texts.append(stemmed_tokens)

> I found that I had the best results with LDA when I additionally did some processing to remove the most common and the rarest words in the corpus
- https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
- 用 *filter_extremes* 函式
- http://blog.acring.me/2018/02/01/Gensim模块处理之dictionary和corpora/

1. 去掉出现次数低于no_below的
2. 去掉出现次数高于no_above的。注意这个小数指的是百分数
3. 在1和2的基础上，保留出现频率前keep_n的单词

In [69]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=3, no_above=0.35)
dictionary.compactify() 
#在执行完前面的过滤操作以后，可能会造成单词的序号之间有空隙，
#这时就可以使用该函数来对词典来进行重新排序，去掉这些空隙。
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=100)

# # 列印前3個topic的詞分布
# print(ldamodel.print_topics(num_topics=3, num_words=3))
# # 列印id為0的topic的詞分布
# print(ldamodel.print_topic(0, topn=5))

In [70]:
# 列印所有topic
for i in range(0, ldamodel.num_topics):
    print(i,'====',ldamodel.print_topic(i))

0 ==== 0.168*"scope" + 0.121*"function" + 0.088*"inventory" + 0.068*"surplus" + 0.054*"operation" + 0.041*"transportation" + 0.027*"supplier" + 0.027*"action" + 0.021*"sale" + 0.021*"delivery"
1 ==== 0.121*"environment" + 0.061*"technology" + 0.061*"opportunity" + 0.061*"issue" + 0.061*"market" + 0.061*"availability" + 0.042*"value" + 0.022*"factor" + 0.022*"lack" + 0.022*"challenge"
2 ==== 0.095*"inventory" + 0.082*"supplier" + 0.055*"role" + 0.055*"capacity" + 0.055*"material" + 0.055*"response" + 0.042*"country" + 0.042*"production" + 0.028*"line" + 0.028*"efficient"
3 ==== 0.013*"use" + 0.013*"technology" + 0.013*"line" + 0.013*"step" + 0.013*"package" + 0.013*"attribute" + 0.013*"volume" + 0.013*"value" + 0.013*"rate" + 0.013*"transportation"
4 ==== 0.013*"opportunity" + 0.013*"mcmaster" + 0.013*"availability" + 0.013*"response" + 0.013*"quality" + 0.013*"purchase" + 0.013*"supplier" + 0.013*"place" + 0.013*"step" + 0.013*"operation"
5 ==== 0.074*"segment" + 0.074*"place" + 0.074*

In [71]:
for i in range(len(doc_set)):
    print(ldamodel.get_document_topics(corpus[i]))

[(5, 0.9727247)]
[(8, 0.99090785)]
[(5, 0.94705456)]
[(2, 0.957139)]
[(0, 0.016666876), (1, 0.016667994), (2, 0.016667137), (3, 0.016666703), (4, 0.016666703), (5, 0.016668413), (6, 0.849994), (7, 0.016666703), (8, 0.016667211), (9, 0.016668238)]
[(9, 0.9816309)]
[(7, 0.9727255)]
[(2, 0.9749977)]
[(9, 0.9709644)]
[(0, 0.011112074), (1, 0.89998823), (2, 0.011112005), (3, 0.011111165), (4, 0.011111165), (5, 0.011112746), (6, 0.011112977), (7, 0.011111164), (8, 0.011114385), (9, 0.011114133)]
[(2, 0.93076295)]
[(0, 0.99217284)]
[(8, 0.96086395)]
[(1, 0.974998)]
[(0, 0.96896034)]


In [72]:
M = len(texts)
num_show_topic = 1  # 每个文档显示前几个主题
doc_topics = ldamodel.get_document_topics(corpus)  # 所有文档的主题分布
idx = np.arange(M)
for i in idx:
    topic = np.array(doc_topics[i])
    topic_distribute = np.array(topic[:, 1])
    # print topic_distribute
    topic_idx = topic_distribute.argsort()[:-num_show_topic - 1:-1]
    print(('doc %d 的主题：' % (i)), topic[topic_idx, 0], topic_distribute[topic_idx])


doc 0 的主题： [5.] [0.97272468]
doc 1 的主题： [8.] [0.99090785]
doc 2 的主题： [5.] [0.94705445]
doc 3 的主题： [2.] [0.95713902]
doc 4 的主题： [6.] [0.84999406]
doc 5 的主题： [9.] [0.98163098]
doc 6 的主题： [7.] [0.97272563]
doc 7 的主题： [2.] [0.9749977]
doc 8 的主题： [9.] [0.97096455]
doc 9 的主题： [1.] [0.89998811]
doc 10 的主题： [2.] [0.93076295]
doc 11 的主题： [0.] [0.99217284]
doc 12 的主题： [8.] [0.96086395]
doc 13 的主题： [1.] [0.974998]
doc 14 的主题： [0.] [0.96895999]
